In [1]:
!python3 --version
#from pyspark import SparkContext
#sc = SparkContext.getOrCreate()
#print(sc.pythonVer)


Python 3.11.9


In [ ]:
#!pip list


In [1]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\paula\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
#!pip uninstall -y pyspark
#pip uninstall -yspark

In [8]:
#!pip cache purge

#!pip install pyspark==3.5.2


In [2]:
#Libreria para análisis EDA
#!pip install matplotlib seaborn sweetviz


In [1]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\paula\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\paula\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
#Importar librerias
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import glob
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import os
import sys

In [4]:

file_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\LOCardReports\Cash_Advance_Transaction-20240903-0921.xls'

# Especificar la ruta del archivo Excel
output_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosSalida\Concatenado2.xlsx'

# Especificar la ruta del archivo Excel
output_path3 = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosSalida\Concatenado3.xlsx'

# Ruta de la carpeta que contiene los archivos de Entrada
folder_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\LOCardReports'
host_folder_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\HostReports'
Logs_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\Logs'


In [5]:
#--------------------------------------------------------------------------LiveOffice Card Reports----------------------------------------------------------------------------------------------

# Obtener una lista de todos los archivos Excel en la carpeta
file_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.xls', '.xlsx'))]

dataframes = []
for file in file_list:
    df = pd.read_excel(file, sheet_name="Sheet2", skiprows=3, header=None)
    file_name = os.path.basename(file)
    new_columns = df.iloc[2].astype(str) + df.iloc[3].astype(str)
    df.columns = new_columns  # Asignar los nuevos encabezados
    df = df[4:]  # Eliminar las filas de encabezado original (0, 1 y 2)
    df.reset_index(drop=True, inplace=True)
    df['file_name'] = os.path.basename(file)# Agregar el nombre del archivo    
    print("Nombre del archivo:", file_name)
    #Tranformación 1 - Nulos
    # Eliminar columnas donde todos los valores son NaN
    print ("Transformacion Nulos\n")
    df = df.dropna(axis=1, how='all')
    #Tranformación 3 - Reemplaza NaN con un valor predeterminado si es necesario
    df = df.fillna(0)    
    # Encuentra el nombre exacto de la columna que contiene 'DATE & TIME'
    target_column = df.filter(like="DATE & TIME").columns[0]
    # Renombra dinámicamente la columna encontrada
    df = df.rename(columns={target_column: "DATE & TIME"})
    df = df.loc[df["DATE & TIME"] != "Totals:"]
    #print(df) 
    if "ATM" in file_name:
        print("El archivo contiene 'ATM' en el nombre.")
        df.insert(loc=df.columns.get_loc("nanSTATUS"), column="FEE($)", value=0)
        #Tranformación 2 - Limpia y Convertir las columnas numéricas a float
        df['DISPENSEDTOTAL($)'] = pd.to_numeric(df['DISPENSEDTOTAL($)'], errors='coerce').fillna(0).astype(float)
        df['TRANSAMOUNT($)'] = pd.to_numeric(df['TRANSAMOUNT($)'], errors='coerce').astype(float)
        df = df.drop(['DISPENSED QTYS$1', 'nan$5', 'nan$10', 'nan$20', 'nan$50', 'nan$100'], axis=1)
        #Tranformación 4-  Convierte las columnas categóricas o de texto a tipo `str`
        df['TRANSTYPE'] = df['TRANSTYPE'].astype(str)        
        df['SEQUENCENUMBER'] = df['SEQUENCENUMBER'].astype(str)        
        df['nanCARD#'] = df['nanCARD#'].astype(str)        
        df['HOSTSEQ NUMBER'] = df['HOSTSEQ NUMBER'].astype(str)
        # limpiar la columna TRANSACTIONTYPE
        df["TRANSTYPE"] = df["TRANSTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSTYPE"] = df["TRANSTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        df = df.rename(columns={"nanSTATUS": "STATUS", "nanCARD#": "CARDNUMBER", "TRANSTYPE": "TRANSACTIONTYPE"})
        print(df) 
    elif "Cash_Advance" in file_name:
        print("El archivo contiene 'Cash_Advance' en el nombre.")  # Agregar el nombre del archivo
        #Tranformación 2 - Limpia y Convertir las columnas numéricas a float
        status_column_index = df.columns.get_loc("FEE($)")
        df['TRANSAMOUNT($)'] = pd.to_numeric(df['TRANSAMOUNT($)'], errors='coerce').astype(float)
        df.insert(loc=status_column_index, column="DISPENSEDTOTAL($)", value=0)
        df.insert(loc=status_column_index, column="COINSTOTAL($)", value=0)        
        #Tranformación 4-  Convierte las columnas categóricas o de texto a tipo `str`
        df['TRANSACTIONTYPE'] = df['TRANSACTIONTYPE'].astype(str)
        df['SEQUENCENUMBER'] = df['SEQUENCENUMBER'].astype(str)        
        df['CARDNUMBER'] = df['CARDNUMBER'].astype(str)        
        df['HOSTSEQ NUMBER'] = df['HOSTSEQ NUMBER'].astype(str)
        # limpiar la columna TRANSACTIONTYPE
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        # Filtrar los registros que no sean 'Credit'
        df = df.loc[df["TRANSACTIONTYPE"] != "Credit"]
        df = df.rename(columns={"nanSTATUS": "STATUS"})
    elif "Debit_Dispense" in file_name:
        print("El archivo contiene 'Debit_Dispense' en el nombre.")  # Agregar el nombre del archivo  
          #Tranformación 2 - Limpia y Convertir las columnas numéricas a float
        status_column_index = df.columns.get_loc("FEE($)")       
        df.insert(loc=status_column_index, column="DISPENSEDTOTAL($)", value=0)
        df.insert(loc=status_column_index, column="COINSTOTAL($)", value=0)        
        #Tranformación 4-  Convierte las columnas categóricas o de texto a tipo `str`
        df['TRANSACTIONTYPE'] = df['TRANSACTIONTYPE'].astype(str)
        df['SEQUENCENUMBER'] = df['SEQUENCENUMBER'].astype(str)        
        df['CARDNUMBER'] = df['CARDNUMBER'].astype(str)        
        df['HOSTSEQ NUMBER'] = df['HOSTSEQ NUMBER'].astype(str)
        # limpiar la columna TRANSACTIONTYPE
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        df = df.rename(columns={"nanSTATUS": "STATUS"})
    elif "Pos_Ticket_Purchase" in file_name:
        print("El archivo contiene 'Pos_Ticket_Purchase' en el nombre.")  # Agregar el nombre del archivo  
          #Tranformación 2 - Limpia y Convertir las columnas numéricas a float
        status_column_index = df.columns.get_loc("FEE($)")       
        df.insert(loc=status_column_index, column="DISPENSEDTOTAL($)", value=0)
        df.insert(loc=status_column_index, column="COINSTOTAL($)", value=0)        
        #Tranformación 4-  Convierte las columnas categóricas o de texto a tipo `str`
        df['TRANSACTIONTYPE'] = df['TRANSACTIONTYPE'].astype(str)
        df['SEQUENCENUMBER'] = df['SEQUENCENUMBER'].astype(str)        
        df['CARDNUMBER'] = df['CARDNUMBER'].astype(str)        
        df['HOSTSEQ NUMBER'] = df['HOSTSEQ NUMBER'].astype(str)
        # limpiar la columna TRANSACTIONTYPE
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        df = df.rename(columns={"nanSTATUS": "STATUS"})
    else:
        print("otra Opcion") 
    dataframes.append(df)

df_combined = pd.concat(dataframes, ignore_index=True)

print("DataFrame combinado con nombres de archivo:")
print(df_combined)


# Guardar el DataFrame de Pandas en Excel
df_combined.to_excel(output_path, index=False, engine='openpyxl') 
print(f"Archivo exportado exitosamente a {output_path}")


Nombre del archivo: ATM_Transaction_Summary-20240903-0919.xls
Transformacion Nulos

El archivo contiene 'ATM' en el nombre.
              DATE & TIME    SEQUENCENUMBER CARDNUMBER HOSTSEQ NUMBER  \
0     08/23/2024 08:13:19  2024082308124108       7816           1452   
1     08/23/2024 08:27:15  2024082308262908       9584           1453   
2     08/23/2024 08:46:09  2024082308453408       4299           1454   
3     08/23/2024 09:06:48  2024082309063308       2610              0   
4     08/23/2024 09:08:19  2024082309074608       6718           1456   
...                   ...               ...        ...            ...   
1194  08/31/2024 03:33:35  2024083103330508       6909           2872   
1195  08/31/2024 04:15:39  2024083104150508       7577           2874   
1196  08/31/2024 04:45:25  2024083104445708       6160              0   
1197  08/31/2024 04:48:35  2024083104480808       1284           2877   
1198  08/31/2024 05:10:02  2024083105092708       5484           2878   


C:\Users\paula\AppData\Local\Temp\ipykernel_45128\4264140334.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
C:\Users\paula\AppData\Local\Temp\ipykernel_45128\4264140334.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
C:\Users\paula\AppData\Local\Temp\ipykernel_45128\4264140334.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('fu

Archivo exportado exitosamente a C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosSalida\Concatenado2.xlsx


In [6]:
#--------------------------------------------------------------------------Host Card Reports----------------------------------------------------------------------------------------------

# Obtener una lista de todos los archivos Excel en la carpeta
file_list = [os.path.join(host_folder_path, f) for f in os.listdir(host_folder_path) if f.endswith(('.xls', '.xlsx'))]

HostDataframes = []
for file in file_list:
    file_name = os.path.basename(file)
 

    if "TransactionLookup" in file_name:        
        df_host = pd.read_excel(file, skiprows=4)
        df_host  = df_host [3:]  # Eliminar las filas de encabezado original (0, 1 y 2)
        df_host.reset_index(drop=True, inplace=True)
        df_host['file_name'] = os.path.basename(file)# Agregar el nombre del archivo    
        print("Nombre del archivo:", file_name)
        df_host['Seq'] = df_host['Seq'].astype(str)
        df_host['PAN'] = df_host['PAN'].astype(str) 
        df_host['Amt. Req']  = df_host['Amt. Req'].str.replace(',', '', regex=False).astype(str) 
        df_host['Amt. Disp'] = df_host['Amt. Disp'].str.replace(',', '', regex=False).astype(str) 
        df_host['Amt. Req']  = pd.to_numeric(df_host['Amt. Req'].str.replace('$', '', regex=False), errors='coerce').astype(float)
        df_host['Amt. Disp'] = pd.to_numeric(df_host['Amt. Disp'].str.replace('$', '', regex=False), errors='coerce').astype(float)
        df_host = df_host.fillna(0)

        print("El archivo contiene TransactionLookup en el nombre.")

    elif "rpttransactiondetailbytid" in file_name:
        df_host = pd.read_excel(file, skiprows=5)
        df_host  = df_host [4:]  # Eliminar las filas de encabezado original (0, 1 y 2)
        df_host.reset_index(drop=True, inplace=True)
        df_host['file_name'] = os.path.basename(file)# Agregar el nombre del archivo    
        print("Nombre del archivo:", file_name)
        print ("Transformacion Nulos\n")
        df_host = df_host.dropna(axis=1, how='all')
        print("El archivo contiene 'rpttransactiondetailbytid' en el nombre.")  # Agregar el nombre del archivo  

    else:
        print("otra Opcion") 
    #df_host = df_host.rename(columns={"CS25DATE & TIME": "DATE & TIME"})
    dataframes.append(df_host)

print("Lectura exitosa")
df_host.to_excel(output_path3, index=False, engine='openpyxl')  # Usa openpyxl como motor
print(f"Archivo exportado exitosamente a {output_path3}")
    

Nombre del archivo: TransactionLookup (8).xls
El archivo contiene TransactionLookup en el nombre.
Lectura exitosa
Archivo exportado exitosamente a C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosSalida\Concatenado3.xlsx


In [7]:
#------------------------------------------------------------------SPARK Consolidado

#import os
#os.environ["PYSPARK_PYTHON"] = "python3"  # O "python" si ese es tu ejecutable
spark = SparkSession.builder.appName("Host_LO Comparative").getOrCreate()

#Pandas a Spark

dfLO = spark.createDataFrame(df_combined)
dfHOST = spark.createDataFrame(df_host)


#Renombre de columnas
dfLO = dfLO.withColumnRenamed("HOSTSEQ NUMBER", "hostseq_number")
dfHOST = dfHOST.withColumnRenamed("Seq", "seq")

# Filtrar el DataFrame dfLO
#dfLO = dfLO.filter(F.col("STATUS") != "Denied")
dfLO = dfLO.filter(F.col("TRANSACTIONTYPE") != "Balance inquiry")


#Verificación
#print('Dataframe de LiveOffice')
#dfLO.printSchema()
#print('Dataframe de Host')
#dfHOST.printSchema()

joined_df = dfLO.join(
    dfHOST,
    dfLO["hostseq_number"] == dfHOST["seq"],
    "left"
)


# Agregar columna de indicador
result_df = joined_df.withColumn(
    "found_in_datastream",
    F.when(F.col("seq").isNotNull(), True).otherwise(False)
)

result_df.show()

#unmatched_df.show()
# Convertir el DataFrame de Spark a Pandas
pandas_df = result_df.toPandas()
pandas_df.to_excel(output_path3, index=False, engine='openpyxl')  # Usa openpyxl como motor
print(f"Archivo exportado exitosamente a {output_path3}")

+-------------------+----------------+----------+--------------+---------------+--------------+-------------+-----------------+------+---------+--------------------+-----------+----+-------+--------------------+----------+-----+----+--------+--------+---------+---------+----------+---------+---------+-----------+---------+------+--------+------+----+--------------------+-------------------+
|        DATE & TIME|  SEQUENCENUMBER|CARDNUMBER|hostseq_number|TRANSACTIONTYPE|TRANSAMOUNT($)|COINSTOTAL($)|DISPENSEDTOTAL($)|FEE($)|   STATUS|           file_name|Terminal ID| seq|TraceID|       Txn Date/Time|Sett  Date|  PAN|Type|FromAcct|Amt. Req|Amt. Disp|Amt. Schg|Amt. Total|Amt. Gtwy|Amt. Ichg|Switch Auth|Gat. Auth|Issuer|     ISO|Status| EMV|           file_name|found_in_datastream|
+-------------------+----------------+----------+--------------+---------------+--------------+-------------+-----------------+------+---------+--------------------+-----------+----+-------+--------------------+-

In [8]:
#-------------------------------------------------------Logs Extraction-------------------------------------------------------------------------------
# Crear una lista para almacenar los DataFrames
logs_dataframes = []


column_names = ["FilteredData"]
df_LogsUnified=  pd.DataFrame()


for subcarpeta in os.listdir(Logs_path):
    ruta_subcarpeta = os.path.join(Logs_path, subcarpeta)
    # Verificar si es una carpeta
    if os.path.isdir(ruta_subcarpeta) and os.path.basename(ruta_subcarpeta).lower() == 'atm':
        print(ruta_subcarpeta)
        for archivo in os.listdir(ruta_subcarpeta):
            if archivo.endswith('.jrn'):  # Filtrar solo archivos con extensión .jrn
                ruta_completa = os.path.join(ruta_subcarpeta, archivo)
                logs_df = pd.read_csv(ruta_completa, sep=';', encoding='utf-8', names=column_names,header=None)  # Ajusta el separador si es necesario
                logs_dataframes.append(logs_df)  
        df_Logscompleto = pd.concat(logs_dataframes, ignore_index=True)   # Concatenar todos los DataFrames en uno solo

        # Paso 1: Filtrar filas que contienen el texto "ATM. 'Posting Transaction Result"
        dfLogsATM = df_Logscompleto[
            df_Logscompleto.apply(lambda row: row.astype(str).str.contains("ATM.     'Posting Transaction Result ").any(), axis=1)
        ]

        # Usar expresiones regulares para extraer los valores
        dfLogsATM['seqNumber'] = dfLogsATM["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
        dfLogsATM['AuthNumber'] = dfLogsATM["FilteredData"].str.extract(r'"AuthNumber":"(\d+)"').astype(str)
        dfLogsATM['CardNumber'] = dfLogsATM["FilteredData"].str.extract(r'"CardNumber":"(\d+)"').astype(str)
        dfLogsATM['HostIP'] = dfLogsATM["FilteredData"].str.extract(r'"HostIP":"([^"]+)"').astype(str)
        dfLogsATM['Amount'] = dfLogsATM["FilteredData"].str.extract(r'"Amount":([0-9]+(?:\.[0-9]+)?)')
        dfLogsATM['DispensedTotal'] = dfLogsATM["FilteredData"].str.extract(r'"DispensedTotal":([0-9]+(?:\.[0-9]+)?)')
        dfLogsATM['TransactionType'] = dfLogsATM["FilteredData"].str.extract(r'"TransactionType":"([^"]+)"').astype(str)
        dfLogsATM['Status'] = dfLogsATM["FilteredData"].str.extract(r'"Status":"([^"]+)"').astype(str)
        dfLogsATM['JournalName'] = (
            dfLogsATM["FilteredData"]
            .str.extract(r'\d+\s+([^\s]+)\s+\'Posting Transaction Result')  # Extraer el texto relevante
            .squeeze()  # Convertir a una Serie si es un DataFrame con una sola columna
            .str.replace(r'\.', '', regex=True)  # Eliminar el punto
        ).astype(str)

        # Agregar df2 debajo de df1
        df_LogsUnified = pd.concat([df_LogsUnified,dfLogsATM], ignore_index=True)

    elif os.path.isdir(ruta_subcarpeta) and os.path.basename(ruta_subcarpeta).lower() == 'cashadvance':
        print(ruta_subcarpeta)
        for archivo in os.listdir(ruta_subcarpeta):
            if archivo.endswith('.jrn'):  # Filtrar solo archivos con extensión .jrn
                ruta_completa = os.path.join(ruta_subcarpeta, archivo)
                logs_df = pd.read_csv(ruta_completa, sep=';', encoding='utf-8', names=column_names,header=None)  # Ajusta el separador si es necesario
                logs_dataframes.append(logs_df)  
        df_Logscompleto = pd.concat(logs_dataframes, ignore_index=True)   # Concatenar todos los DataFrames en uno solo

        # Paso 1: Filtrar filas que contienen el texto "ATM. 'Posting Transaction Result"
        dfLogsCA = df_Logscompleto[
            df_Logscompleto.apply(lambda row: row.astype(str).str.contains("Cash Advance Transaction Info. 'Posting Transaction Result").any(), axis=1)
        ]
        
        # Usar expresiones regulares para extraer los valores
        dfLogsCA['seqNumber'] = dfLogsCA["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
        dfLogsCA['AuthNumber'] = dfLogsCA["FilteredData"].str.extract(r'"AuthNumber":"(\d+)"').astype(str)
        dfLogsCA['CardNumber'] = dfLogsCA["FilteredData"].str.extract(r'"CardNumber":"(\d+)"').astype(str)
        dfLogsCA['HostIP'] = dfLogsCA["FilteredData"].str.extract(r'"HostIP":"([^"]+)"').astype(str)
        dfLogsCA['Amount'] = dfLogsCA["FilteredData"].str.extract(r'"Amount":([0-9]+(?:\.[0-9]+)?)')
        dfLogsCA['DispensedTotal'] = dfLogsCA["FilteredData"].str.extract(r'"DispensedTotal":([0-9]+(?:\.[0-9]+)?)')
        dfLogsCA['TransactionType'] = dfLogsCA["FilteredData"].str.extract(r'"TransactionType":"([^"]+)"').astype(str)
        dfLogsCA['Status'] = dfLogsCA["FilteredData"].str.extract(r'"Status":"([^"]+)"').astype(str)
        dfLogsCA['JournalName'] = (
            dfLogsCA["FilteredData"]
            .str.extract(r'------------>\s*(.*?)\s*Transaction Info\.')  # Extraer el texto relevante
            .squeeze()  # Convertir a una Serie si es un DataFrame con una sola columna
            .str.replace(r'\.', '', regex=True)  # Eliminar el punto
        ).astype(str)

        # Agregar df2 debajo de df1
        df_LogsUnified = pd.concat([df_LogsUnified,dfLogsCA], ignore_index=True)


print(df_LogsUnified)

df_LogsUnified.to_excel(output_path, index=False, engine='openpyxl')  # Usa openpyxl como motor
print(f"Archivo exportado exitosamente a {output_path}")

C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\Logs\ATM


C:\Users\paula\AppData\Local\Temp\ipykernel_45128\3279142524.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsATM['seqNumber'] = dfLogsATM["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_45128\3279142524.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsATM['AuthNumber'] = dfLogsATM["FilteredData"].str.extract(r'"AuthNumber":"(\d+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_45128\3279142524.py:30: SettingWithCopyW

C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\Logs\CASHADVANCE


C:\Users\paula\AppData\Local\Temp\ipykernel_45128\3279142524.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsCA['seqNumber'] = dfLogsCA["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_45128\3279142524.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsCA['AuthNumber'] = dfLogsCA["FilteredData"].str.extract(r'"AuthNumber":"(\d+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_45128\3279142524.py:63: SettingWithCopyWarni

                                           FilteredData         seqNumber  \
0     ___00:09:12 403002 ATM.     'Posting Transacti...  2024082300085508   
1     ___00:18:28 403002 ATM.     'Posting Transacti...  2024082300181108   
2     ___00:19:37 403002 ATM.     'Posting Transacti...  2024082300190208   
3     ___00:20:40 403002 ATM.     'Posting Transacti...  2024082300201108   
4     ___00:32:37 403002 ATM.     'Posting Transacti...  2024082300320708   
...                                                 ...               ...   
1526  ___21:08:27 401000 ------------>Cash Advance T...  0831202421074647   
1527  ___21:10:31 401000 ------------>Cash Advance T...  0831202421095407   
1528  ___22:02:50 401000 ------------>Cash Advance T...  0831202422023934   
1529  ___22:23:17 401000 ------------>Cash Advance T...  0831202422223273   
1530  ___23:17:40 401000 ------------>Cash Advance T...  0831202423163985   

     AuthNumber CardNumber   HostIP Amount DispensedTotal TransactionType  

In [9]:
#------------------------------------------------------------------SPARK Consolidado logs

spark = SparkSession.builder.appName("Logs_LO_Host Comparative").getOrCreate()

#Pandas a Spark

dfLogs = spark.createDataFrame(df_LogsUnified)
dfLOHOST =(result_df)


#Renombre de columnas
#dfLO = dfLO.withColumnRenamed("HOSTSEQ NUMBER", "hostseq_number")
#dfHOST = dfHOST.withColumnRenamed("Seq", "seq")

# Filtrar el DataFrame dfLO
#dfLO = dfLO.filter(F.col("STATUS") != "Denied")
#dfLO = dfLO.filter(F.col("TRANSACTIONTYPE") != "Balance inquiry")


#Verificación
#print('Dataframe de LiveOffice')
#dfLO.printSchema()
#print('Dataframe de Host')
#dfHOST.printSchema()

#-Join Logs con LOHOSTUNIFIED
joined_df = dfLogs.join(
    dfLOHOST,
    dfLogs["seqNumber"] == dfLOHOST["SEQUENCENUMBER"],
    "left"
)


# Agregar columna de indicador
result_df2 = joined_df.withColumn(
    "found_in_Systems",
    F.when(F.col("seq").isNotNull(), True).otherwise(False)
)

result_df2.show()

#-Join Logs con HOST
joined_df = dfLogs.join(
    dfHOST,
    dfLogs["AuthNumber"] == dfHOST["seq"],
    "left"
)


# Agregar columna de indicador
result_df3 = joined_df.withColumn(
    "found_in_datastream",
    F.when(F.col("seq").isNotNull(), True).otherwise(False)
)

result_df3.show()

pandas_df1 = result_df2.toPandas()
pandas_df2 = result_df3.toPandas()

with pd.ExcelWriter(output_path3, engine='openpyxl') as writer:
    pandas_df1.to_excel(writer, sheet_name='Hoja1', index=False)  # Guardar df1 en 'Hoja1'
    pandas_df2.to_excel(writer, sheet_name='Hoja2', index=False)  # Guardar df2 en 'Hoja2'



#pandas_df.to_excel(output_path3, index=False, engine='openpyxl')  # Usa openpyxl como motor
print(f"Archivo exportado exitosamente a {output_path3}")

+--------------------+----------------+----------+----------+-------+------+--------------+---------------+---------+-----------+-------------------+----------------+----------+--------------+---------------+--------------+-------------+-----------------+------+---------+--------------------+-----------+----+-------+--------------------+----------+-----+----+--------+--------+---------+---------+----------+---------+---------+-----------+---------+------+--------+------+----+--------------------+-------------------+----------------+
|        FilteredData|       seqNumber|AuthNumber|CardNumber| HostIP|Amount|DispensedTotal|TransactionType|   Status|JournalName|        DATE & TIME|  SEQUENCENUMBER|CARDNUMBER|hostseq_number|TRANSACTIONTYPE|TRANSAMOUNT($)|COINSTOTAL($)|DISPENSEDTOTAL($)|FEE($)|   STATUS|           file_name|Terminal ID| seq|TraceID|       Txn Date/Time|Sett  Date|  PAN|Type|FromAcct|Amt. Req|Amt. Disp|Amt. Schg|Amt. Total|Amt. Gtwy|Amt. Ichg|Switch Auth|Gat. Auth|Issuer|  

In [ ]:
#--------------------------------------------------------------------------Análisis EDA----------------------------------------------------------------------------------------------
import pandas as pd
import sweetviz as sv
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar archivo Excel
df= pd.read_excel(file_path, sheet_name="Sheet2", skiprows=2, header=None) # Cambia por la ruta de tu archivo
new_columns = df.iloc[2].astype(str) + df.iloc[3].astype(str)
df.columns = new_columns  # Asignar los nuevos encabezados
df = df[4:]  # Eliminar las filas de encabezado original (0, 1 y 2)
df.reset_index(drop=True, inplace=True)
#df['file_name'] = os.path.basename(file)# Agregar el nombre del archivo    
#print("Nombre del archivo:", file_name)
    #Tranformación 1 - Nulos
    # Eliminar columnas donde todos los valores son NaN
print ("Transformacion Nulos\n")
df = df.dropna(axis=1, how='all')
    #Tranformación 3 - Reemplaza NaN con un valor predeterminado si es necesario
df = df.fillna(0)
#df = df.loc[df["CS25DATE & TIME"] != "Totals:"]
# Obtener información básica sobre el DataFrame
df.info()

# Resumen estadístico de las columnas numéricas
print(df.describe())

# Verificar valores nulos
print(df.isnull().sum())



# Histograma para todas las columnas numéricas
df.hist(bins=20, figsize=(10, 8))
plt.show()

# Boxplot para una columna específica (ejemplo: columna "TRANSACTIONTYPE")
sns.boxplot(x=df["nanCARD"])
plt.show()

# Mapa de calor de correlación
corr_matrix = df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.show()

# Gráfico de barras para una columna categórica (ejemplo: "categoría")
sns.countplot(x="nanTRANS", data=df)
plt.show()
# Generar el reporte
#reporte = sv.analyze(data)

# Guardar el reporte
#reporte.show_html("reporte_sweetviz.html")
